In [1]:
import numpy as np
import tensorflow as tf

In [3]:
"""
https://stackoverflow.com/questions/44395715/how-tf-nn-dynamic-rnn-works-with-different-lengths-of-inputs-sepcially-when-work
"""
# Create input data
X = np.random.randn(2, 10, 8)
print(X.shape)
print(X)
# The second example is of length 6 
X[1,6:] = 0

print(X.shape)
print(X)

X_lengths = [10, 6]

cell = tf.nn.rnn_cell.LSTMCell(num_units=64, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

print(outputs)
print(last_states)

(2, 10, 8)
[[[-1.25551353  2.24639614  1.77892508 -0.86140806 -2.07226145
   -0.25837322 -0.79183211  1.27295041]
  [ 0.22711508 -0.03995453  0.26857762  0.20019934  0.19614549
   -0.56601487  0.48734301 -0.86131581]
  [ 0.73954955  1.19888853  0.10539584 -1.1733565  -1.73790008
    1.40931994  0.44022291 -0.86081129]
  [-1.15937223  0.18712682 -1.51621826  0.79610983 -0.32961062
   -0.38010997 -0.10635119 -0.37332707]
  [ 0.88461698  0.60936157 -0.57903523  1.23776283  0.66189392
    0.40976684  0.60476817 -0.54422679]
  [ 0.48779974 -0.20903999 -0.5431919   1.05902886  0.30600349
    0.50956036 -0.02103815  0.71842879]
  [ 0.14855874  1.20255152 -1.67059573 -0.75080575  0.04404576
    1.29235602 -0.50527332  0.74676505]
  [-0.81663574 -0.95567546 -0.95887213  1.49910786  1.42503677
    0.30682337 -0.72888977 -0.92867031]
  [-1.22595043  0.55791177 -0.15910561  1.12730604  0.58292168
    0.4703978  -1.23886757  0.96133241]
  [ 0.12007376 -0.85622867  0.18141394 -0.10204211  0.85941282

In [7]:
import tensorflow as tf    
#from tensorflow.models.rnn import rnn  
#from tensorflow.models.rnn.rnn_cell import BasicLSTMCell, LSTMCell    
import numpy as np

if __name__ == '__main__':
  np.random.seed(1)      
  size = 100
  batch_size= 100
  n_steps = 45
  seq_width = 50     

  initializer = tf.random_uniform_initializer(-1,1) 

  seq_input = tf.placeholder(tf.float32, [n_steps, batch_size, seq_width])
    #sequence we will provide at runtime  
  early_stop = tf.placeholder(tf.int32)
    #what timestep we want to stop at

  inputs = [tf.reshape(i, (batch_size, seq_width)) for i in tf.split(n_steps, seq_input, 0)]
    #inputs for rnn needs to be a list, each item being a timestep. 
    #we need to split our input into each timestep, and reshape it because split keeps dims by default  

  cell = tf.nn.rnn_cell.BasicLSTMCell(size, seq_width, initializer=initializer)  
  initial_state = cell.zero_state(batch_size, tf.float32)
  #outputs, states = rnn.rnn(cell, inputs, initial_state=initial_state, sequence_length=early_stop) #tf.nn.rnn_cell.BasicLSTMCell
  outputs, states = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, sequence_length=early_stop) #
    #set up lstm

  iop = tf.initialize_all_variables()
    #create initialize op, this needs to be run by the session!
  session = tf.Session()
  session.run(iop)
    #actually initialize, if you don't do this you get errors about uninitialized stuff

  feed = {early_stop:100, seq_input:np.random.rand(n_steps, batch_size, seq_width).astype('float32')}
    #define our feeds. 
    #early_stop can be varied, but seq_input needs to match the shape that was defined earlier

  outs = session.run(outputs, feed_dict=feed)
    #run once
    #output is a list, each item being a single timestep. Items at t>early_stop are all 0s
  print(type(outs))
  print(len(outs))

TypeError: Value passed to parameter 'size_splits' has DataType float32 not in list of allowed values: int32, int64